In [1]:
from transformers import PullData

form_window = 24
target_window = 4

GetData = PullData()

GetData.fit(ticker = 'nvda',
            start_date ='1980-01-01',
            end_date = '2022-06-11',
            interval = '1wk',
            progress = False,
            form_window = form_window,
            target_window = target_window,
            timeperiod1 = 6,
            timeperiod2 = 12,
            timeperiod3 = 24,
            )

data_prep = GetData.transform()
 
print(data_prep.shape)
print("hello")
print("hello")
data_prep.head(25)


(29825, 8)
hello
hello


,Date,Open,High,Low,Close,EMA6,EMA12,EMA24
0,1999-01-18 00:00:00,0.437500,0.488281,0.388021,0.410156,0.424262,0.428928,0.399034
1,1999-01-25 00:00:00,0.442708,0.467448,0.395833,0.395833,0.424262,0.428928,0.399034
2,1999-02-01 00:00:00,0.395833,0.416667,0.360677,0.412760,0.424262,0.428928,0.399034
3,1999-02-08 00:00:00,0.415365,0.437500,0.372396,0.434896,0.424262,0.428928,0.399034
4,1999-02-15 00:00:00,0.442708,0.460938,0.393229,0.434896,0.424262,0.428928,0.399034
5,1999-02-22 00:00:00,0.442708,0.546875,0.414063,0.457031,0.424262,0.428928,0.399034
6,1999-03-01 00:00:00,0.468750,0.479167,0.411458,0.438802,0.428416,0.428928,0.399034
7,1999-03-08 00:00:00,0.427083,0.466146,0.416667,0.424479,0.427291,0.428928,0.399034
8,1999-03-15 00:00:00,0.434896,0.450521,0.420573,0.437500,0.430208,0.428928,0.399034
9,1999-03-22 00:00:00,0.446615,0.447917,0.380208,0.436198,0.431920,0.428928,0.399034


In [3]:
window_size = form_window + 1 #(24+1)

batch_size = 64 #Train batchsize
batch_valid = 8 #Valid batchsize

tf.random.set_seed(7788)
np.random.seed(7788)



class NormalizeData(BaseEstimator, TransformerMixin):
    """
    """

    def __init__(self):
        """Initialize class with two parameters
        """
        super().__init__()
        self.ticker = None
        self.start_date = None

 

    def fit(self, ticker: str, start_date: str):
        # Data pulling
        self.ticker = ticker
        self.start_date = start_date


        return self

    def transform(self):
        """

        """


        return final_df_w


In [ ]:
def Normalize(df,window_size):
    """
    Get dataframe and window_size as an input
    
    Normalize data based on linear scaling formula
    
    save maxv and minv of each window for future reversion
    
    """
    
    print("\nNormalize dataframe \n")
    
    #Initialize dataitems
    counter = 0
    inc = 0
    Highs = []
    Lows = []

    df_norm = pd.DataFrame()

    #Loop through each window separately to normalize
    for row in range(0,len(df),window_size):
        
        counter += 1
        df_temp = pd.DataFrame()


        #Get maxv and minv of window
        while inc < df.shape[1]: 

            #Get max High
            if row + inc<len(df):
                
                Highs.append(df.iloc[row+inc][1])
                Lows.append(df.iloc[row+inc][2])

                inc += 1
            else:
                break

        #reset inc
        inc = 0

        #Save Max and Min
        maxv = max(Highs)
        minv = min(Lows)
        
        #testing
        maxv = np.max(df.iloc[row:row + window_size,:].to_numpy())
        minv = np.min(df.iloc[row:row + window_size,:].to_numpy())
        #print(maxv)
        #print(minv)
        #break
        #Reset 
        Highs = []
        Lows = []

        #Print first 2 windows for checking
        if counter < 3:
            #Print data windowing
            print("\nWindow:" + str(counter) + "\n " + str(df.iloc[row:row + window_size,:]))
            print("\nMax value is ", maxv)
            print("Min value is ", minv)
            print("\n Normalized:\n "+ str((df.iloc[row:row + window_size,:]-minv)/(maxv-minv)))

        #Merge normalized window to new dataframe
        df_temp = (df.iloc[row:row + window_size,:]-minv)/(maxv-minv)
        df_temp['maxv'] = maxv
        df_temp['minv'] = minv

        df_norm = pd.concat([df_norm,df_temp],axis=0)     

    print("\nDone")
    
    return df_norm
    
df_norm = Normalize(initial_df,window_size)